In [ ]:
# TODO UPDATE THIS OLD MESSY SCRIPT FROM RA

In [220]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math
import scipy
from scipy.stats import kurtosis
from scipy.stats import skew
from datetime import timedelta

In [ ]:
# read in the data
panel_df = pd.read_pickle("../3-data/clean/raw_panel.pkl")


In [222]:
# data frame for daily frequency
daily_df = panel_df.copy()

In [223]:
# create the data frame for weekly frequency where only sundays are kept
panel_df['weekday'] = panel_df['date'].dt.day_of_week
weekly_df = panel_df[panel_df.weekday == 6]

In [224]:
# create data frame with monthly frequency where we define months as every 28 days
weekly_df.assign(week_count=weekly_df.groupby(['date']).ngroup() + 1)
temp_df = weekly_df[['date']].drop_duplicates().reset_index(drop=True)
temp_df['week_count'] = np.arange(temp_df.shape[0])
weekly_df = weekly_df.merge(temp_df,
                            on=['date'],
                            how='inner',
                            validate='many_to_one')
monthly_df = weekly_df[0 == (weekly_df.week_count % 4)]

In [225]:
# compute daily returns for each asset
temp_daily_df = daily_df.copy()
temp_daily_df.rename(columns={'usd_per_token':'usd_per_token_1'},inplace=True)
temp_daily_df = temp_daily_df[["date","asset","usd_per_token_1"]]
temp_daily_df["date"] = temp_daily_df["date"] + timedelta(days=1)
daily_df = temp_daily_df.merge(daily_df,
                               on=["date", "asset"],
                               how='inner',
                               validate='one_to_one')
daily_df['simpleret'] = (daily_df.usd_per_token-daily_df.usd_per_token_1)/daily_df.usd_per_token_1
daily_df = daily_df.drop('usd_per_token_1', axis=1)

In [226]:
# compute the weekly returns for each asset
temp_weekly_df = weekly_df.copy()
temp_weekly_df.rename(columns={'usd_per_token':'usd_per_token_1'},inplace=True)
temp_weekly_df = temp_weekly_df[["date","asset","usd_per_token_1"]]
temp_weekly_df["date"] = temp_weekly_df["date"] + timedelta(days=7)
weekly_df = temp_weekly_df.merge(weekly_df,
                            on=["date", "asset"],
                            how='inner',
                            validate='one_to_one')
weekly_df['simpleret'] = (weekly_df.usd_per_token-weekly_df.usd_per_token_1)/weekly_df.usd_per_token_1
weekly_df = weekly_df.drop('usd_per_token_1', axis=1)

In [227]:
# compute the monthly returns for each asset
temp_monthly_df = monthly_df.copy()
temp_monthly_df.rename(columns={'usd_per_token':'usd_per_token_1'},inplace=True)
temp_monthly_df = temp_monthly_df[["date","asset","usd_per_token_1"]]
temp_monthly_df["date"] = temp_monthly_df["date"] + timedelta(days=28)
monthly_df = temp_monthly_df.merge(monthly_df,
                            on=["date", "asset"],
                            how='inner',
                            validate='one_to_one')
monthly_df['simpleret'] = (monthly_df.usd_per_token-monthly_df.usd_per_token_1)/monthly_df.usd_per_token_1
monthly_df = monthly_df.drop('usd_per_token_1', axis=1)

In [228]:
# create a new data with daily frequenct frame called "new_daily_df" where only data from 2016-2021 are kept
daily_df["date"] = pd.to_datetime(daily_df.date)
new_daily_df = daily_df[daily_df.date.dt.year != 2015]

In [229]:
# create a new data frame with weekly frequency called "new_weekly_df" where only data from 2016-2021 are kept
weekly_df["date"] = pd.to_datetime(weekly_df.date)
new_weekly_df = weekly_df[weekly_df.date.dt.year != 2015]

In [230]:
# create a new data frame with monthly frequency called "new_monthly_df" where only data from 2016-2021 are kept
monthly_df["date"] = pd.to_datetime(monthly_df.date)
new_monthly_df = monthly_df[monthly_df.date.dt.year != 2015]

In [ ]:
# recreate the 3 side-by-side histograms for daily, weekly, and monthly simple returns 

plt.subplot(1,3,1)
plt.hist(new_daily_df.simpleret, bins = [-1,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
plt.xlim(-1,1)
plt.xlabel("daily simple returns")

plt.subplot(1,3,2)
plt.hist(new_weekly_df.simpleret, bins = [-1,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2])
plt.xlim(-1,2)
plt.xlabel("weekly simple returns")

plt.subplot(1,3,3)
plt.hist(new_monthly_df.simpleret, bins = [-1,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4,4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,5,5.1,5.2,5.3,5.4,5.5,5.6,5.7,5.8,5.9,6])
plt.xlim(-1,6)
plt.xlabel("monthly simple returns")

In [ ]:
# recreate the daily simple returns histogram alone (some extremely positive returns (outliers) might not be captured but might affect the distribution)
plt.hist(new_daily_df.simpleret, bins = [-1,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
plt.xlim(-1,1)
plt.xlabel("daily simple returns")

In [ ]:
# recreate the weekly simple returns histogrm alone (some extremely positive returns (outliers) might not be captured but might affect the distribution)
plt.hist(new_weekly_df.simpleret, bins = [-1,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2])
plt.xlim(-1,2)
plt.xlabel("weekly simple returns")

In [ ]:
# recreate the monthly simple returns histogrm alone (some extremely positive returns (outliers) might not be captured but might affect the distribution)
plt.hist(new_monthly_df.simpleret, bins = [-1,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.1,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4,4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9,5,5.1,5.2,5.3,5.4,5.5,5.6,5.7,5.8,5.9,6])
plt.xlim(-1,6)
plt.xlabel("monthly simple returns")

In [ ]:
# create the within day total market cap for each day
new_daily_df['mcap_sum'] = new_daily_df.groupby('date')['usd_mcap'].transform('sum')

In [ ]:
# compute the market cap share of each asset on each day
new_daily_df['mcap_weight'] = new_daily_df.usd_mcap / new_daily_df.mcap_sum

In [ ]:
# calculate the within day average return where each asset's return is weighted by its market share
new_daily_df['mcap_r_t'] = new_daily_df.simpleret * new_daily_df.mcap_weight
new_daily_df['mcap_r_t'] = new_daily_df.groupby('date')[['mcap_r_t']].transform(sum)


In [ ]:
# repeat the same procedure to create a data frame that computes the within week average return where each assets return is weighted by its market share

new_weekly_df['mcap_sum'] = new_weekly_df.groupby('date')['usd_mcap'].transform('sum')
new_weekly_df['mcap_weight'] = new_weekly_df.usd_mcap / new_weekly_df.mcap_sum
new_weekly_df['mcap_r_t'] = new_weekly_df.simpleret * new_weekly_df.mcap_weight
new_weekly_df['mcap_r_t'] = new_weekly_df.groupby('date')[['mcap_r_t']].transform(sum)
new_weekly_df = new_weekly_df[['date','mcap_r_t']].drop_duplicates()

In [ ]:
# repeat the same procedure to create a data frame that computes the within month average return where each assets return is weighted by its market share

new_monthly_df['mcap_sum'] = new_monthly_df.groupby('date')['usd_mcap'].transform(sum)
new_monthly_df['mcap_weight'] = new_monthly_df.usd_mcap / new_monthly_df.mcap_sum
new_monthly_df['mcap_r_t'] = new_monthly_df.simpleret * new_monthly_df.mcap_weight
new_monthly_df['mcap_r_t'] = new_monthly_df.groupby('date')[['mcap_r_t']].transform(sum)
new_monthly_df = new_monthly_df[['date','mcap_r_t']].drop_duplicates()

In [ ]:
new_daily_df["one_plus_simpleret"] = new_daily_df["simpleret"]+1
new_daily_df["one_plus_mcap_rt"] = new_daily_df["mcap_r_t"]+1

new_daily_df["daily_val_investment"] = new_daily_df.groupby('date')['one_plus_simpleret'].cumprod() # cumulative product of all daily simple returns

new_daily_df["bitcoin_return"] = new_daily_df.simpleret[new_daily_df.asset=="bitcoin"]
new_daily_df["one_plus_bitcoin_return"] = new_daily_df["bitcoin_return"].fillna(0)+1
new_daily_df["bitcoin_cum_ret"] = new_daily_df.groupby('date')['one_plus_bitcoin_return'].cumprod() # cumulative product of daily simple returns of bitcoin

new_daily_df["ethereum_return"] = new_daily_df.simpleret[new_daily_df.asset=="ethereum"]
new_daily_df["one_plus_ethereum_return"] = new_daily_df["ethereum_return"].fillna(0)+1
new_daily_df["ethereum_cum_ret"] = new_daily_df.groupby('date')['one_plus_ethereum_return'].cumprod() # cumultaive product of daily simple returns of ethereum

new_daily_df["CMKT_val_investment"] = new_daily_df.groupby('date')['one_plus_mcap_rt'].cumprod() # cumulative product of daily market weighted simple returns

In [241]:
bitcoin_df = new_daily_df[new_daily_df.asset == "bitcoin"]
btc_cum_inv_array = bitcoin_df['one_plus_simpleret'].cumprod().values


In [242]:
eth_df = new_daily_df[new_daily_df.asset == "ethereum"]
eth_cum_inv_array = eth_df['one_plus_simpleret'].cumprod().values

In [243]:
cmkt_df = new_daily_df[['date','one_plus_mcap_rt']]
cmkt_df = cmkt_df.drop_duplicates()
cmkt_cum_inv_array = cmkt_df['one_plus_mcap_rt'].cumprod().values

In [ ]:
dates = bitcoin_df.date.values
cum_inv_df = pd.DataFrame(data={'Bitcoin':btc_cum_inv_array,
                   'Ethereum':eth_cum_inv_array,
                   'Crypto Market':cmkt_cum_inv_array})
cum_inv_df = cum_inv_df.set_index(dates)
cum_inv_df.plot(style = ['b-','g--','r-.'])


In [245]:
VTI = pd.read_csv("../3-data/raw/VTI.csv")
VTI_df = pd.DataFrame(data={'Date':VTI['Date'],
                            'Price':VTI['Adj Close']})
VTI_df['Date'] = pd.to_datetime(VTI_df.Date)
VTI_df         = VTI_df[VTI_df.Date.dt.year >= 2016]
VTI_df         = VTI_df[VTI_df.Date.dt.year <= 2021]
VTI_df = VTI_df.reset_index(drop=True)
VTI_df['returns'] = VTI_df['Price'].pct_change(1)
VTI_df = VTI_df.drop(index=0)
VTI_mean = statistics.mean(VTI_df.returns)
VTI_sd = statistics.stdev(VTI_df.returns)
VTI_t_stat = VTI_mean / (VTI_sd/math.sqrt(1510))
VTI_sharpe = VTI_mean / VTI_sd
VTI_skewness = skew(VTI_df.returns)
VTI_kurt = kurtosis(VTI_df.returns,fisher = False)
VTI_positive = len(VTI_df.loc[VTI_df.returns>0]) / 1510

cmkt_df['mcap_rt'] = cmkt_df['one_plus_mcap_rt']-1
cmkt_mean = statistics.mean(cmkt_df.mcap_rt)
cmkt_sd = statistics.stdev(cmkt_df.mcap_rt)
cmkt_t_stat = cmkt_mean / (cmkt_sd/math.sqrt(2192))
cmkt_sharpe = cmkt_mean / cmkt_sd
cmkt_skewness = skew(cmkt_df.mcap_rt)
cmkt_kurt = kurtosis(cmkt_df.mcap_rt,fisher = False)
cmkt_positive = len(cmkt_df.loc[cmkt_df.mcap_rt>0]) / 2192

cmkt_disaster_5 = len(cmkt_df.loc[cmkt_df.mcap_rt<-0.05]) 
cmkt_disaster_5_percent = cmkt_disaster_5 / 2192                                        
cmkt_disaster_10 = len(cmkt_df.loc[cmkt_df.mcap_rt<-0.1])
cmkt_disaster_10_percent = cmkt_disaster_10/2192
cmkt_disaster_15 = len(cmkt_df.loc[cmkt_df.mcap_rt<-0.15])
cmkt_disaster_15_percent = cmkt_disaster_15/2192
cmkt_disaster_20 = len(cmkt_df.loc[cmkt_df.mcap_rt<-0.2])
cmkt_disaster_20_percent = cmkt_disaster_20/2192
cmkt_miracle_5 = len(cmkt_df.loc[cmkt_df.mcap_rt>0.05]) 
cmkt_miracle_5_percent = cmkt_miracle_5 / 2192
cmkt_miracle_10 = len(cmkt_df.loc[cmkt_df.mcap_rt>0.1]) 
cmkt_miracle_10_percent = cmkt_miracle_10 / 2192
cmkt_miracle_15 = len(cmkt_df.loc[cmkt_df.mcap_rt>0.15]) 
cmkt_miracle_15_percent = cmkt_miracle_15 / 2192
cmkt_miracle_20 = len(cmkt_df.loc[cmkt_df.mcap_rt>0.2]) 
cmkt_miracle_20_percent = cmkt_miracle_20 / 2192

bitcoin_mean = statistics.mean(bitcoin_df.simpleret)
bitcoin_sd = statistics.stdev(bitcoin_df.simpleret)
bitcoin_t_stat = bitcoin_mean / (bitcoin_sd/math.sqrt(2192))
bitcoin_sharpe = bitcoin_mean / bitcoin_sd
bitcoin_skewness = skew(bitcoin_df.simpleret)
bitcoin_kurt = kurtosis(bitcoin_df.simpleret,fisher = False)
bitcoin_positive = len(bitcoin_df.loc[bitcoin_df.simpleret>0]) / 2192

ethereum_mean = statistics.mean(eth_df.simpleret)
ethereum_sd = statistics.stdev(eth_df.simpleret)
ethereum_t_stat = ethereum_mean / (ethereum_sd/math.sqrt(2192))
ethereum_sharpe = ethereum_mean / ethereum_sd
ethereum_skewness = skew(eth_df.simpleret)
ethereum_kurt = kurtosis(eth_df.simpleret,fisher = False)
ethereum_positive = len(eth_df.loc[eth_df.simpleret>0]) / 2192


In [246]:
Extreme_events_df = pd.DataFrame(data={"Disasters":("< -5%","< -10%","< -15%","< -20%"),
                                      "Disaster_Counts":(cmkt_disaster_5,cmkt_disaster_10,cmkt_disaster_15,cmkt_disaster_20),
                                      "Disaster_Proportion":(cmkt_disaster_5_percent,cmkt_disaster_10_percent,cmkt_disaster_15_percent,cmkt_disaster_20_percent),
                                      "Miracles":("> 5%","> 10%","> 15%","> 20%"),
                                      "Miracle_Counts":(cmkt_miracle_5,cmkt_miracle_10,cmkt_miracle_15,cmkt_miracle_20),
                                      "Miracle_Proportions":(cmkt_miracle_5_percent,cmkt_miracle_10_percent,cmkt_miracle_15_percent,cmkt_miracle_20_percent)})

In [247]:
daily_stats_df = pd.DataFrame(data={"daily":("CMKT","Bitcoin","Ethereum","Stock"),
                                    "mean":(cmkt_mean,bitcoin_mean,ethereum_mean,VTI_mean),
                                  "sd":(cmkt_sd,bitcoin_sd,ethereum_sd,VTI_sd),
                                  "t_stat":(cmkt_t_stat,bitcoin_t_stat,ethereum_t_stat,VTI_t_stat),
                                  "sharpe":(cmkt_sharpe,bitcoin_sharpe,ethereum_sharpe,VTI_sharpe),
                                  "skewness":(cmkt_skewness,bitcoin_skewness,ethereum_skewness,VTI_skewness),
                                  "kurtosis":(cmkt_kurt,bitcoin_kurt,ethereum_kurt,VTI_kurt),
                                  "% > 0":(cmkt_positive,bitcoin_positive,ethereum_positive,VTI_positive)
                                  })

In [ ]:
new_VTI = pd.read_csv("../3-data/raw/VTI.csv")
new_VTI_df = pd.DataFrame(data={'Date':VTI['Date'],
                                'Price':VTI['Adj Close']})
VTI_df['Date'] = pd.to_datetime(VTI_df.Date)
VTI_df         = VTI_df[VTI_df.Date.dt.year >= 2016]
VTI_df         = VTI_df[VTI_df.Date.dt.year <= 2021]
VTI_df = VTI_df.reset_index(drop=True)
new_VTI_df['weekday'] = \
pd.to_datetime(new_VTI_df['Date']).dt.day_of_week
VTI_df_weekly = new_VTI_df[new_VTI_df.weekday == 0]
VTI_df_weekly['day_diff'] = (pd.to_datetime(VTI_df_weekly.Date)-
                             pd.to_datetime(VTI_df_weekly.Date).shift(1))[1:].astype('timedelta64[D]').astype(int)
VTI_df_weekly = VTI_df_weekly[VTI_df_weekly.day_diff==7]

VTI_df_weekly['weekly_return'] = VTI_df_weekly['Price'].pct_change(1)
new_weekly_VTI_df = VTI_df_weekly.reset_index()[1:]
VTI_mean_weekly = statistics.mean(new_weekly_VTI_df.weekly_return)
VTI_sd_weekly = statistics.stdev(new_weekly_VTI_df.weekly_return)
VTI_t_stat_weekly = VTI_mean_weekly / (VTI_sd_weekly/math.sqrt(253))
VTI_sharpe_weekly = VTI_mean_weekly / VTI_sd_weekly
VTI_skewness_weekly = skew(new_weekly_VTI_df.weekly_return)
VTI_kurt_weekly = kurtosis(new_weekly_VTI_df.weekly_return,fisher = False)
VTI_positive_weekly = len(new_weekly_VTI_df.loc[new_weekly_VTI_df.weekly_return>0]) / 253

weekly_bitcoin_df = bitcoin_df[bitcoin_df.date.dt.weekday == 6]
weekly_bitcoin_df['weekly_return'] = weekly_bitcoin_df['usd_per_token'].pct_change(1)
new_weekly_bitcoin_df = weekly_bitcoin_df.reset_index()[1:]
bitcoin_mean_weekly = statistics.mean(new_weekly_bitcoin_df.weekly_return)
bitcoin_sd_weekly = statistics.stdev(new_weekly_bitcoin_df.weekly_return)
bitcoin_t_stat_weekly = bitcoin_mean_weekly / (bitcoin_sd_weekly/math.sqrt(312))
bitcoin_sharpe_weekly = bitcoin_mean_weekly / bitcoin_sd_weekly
bitcoin_skewness_weekly = skew(new_weekly_bitcoin_df.weekly_return)
bitcoin_kurt_weekly = kurtosis(new_weekly_bitcoin_df.weekly_return,fisher = False)
bitcoin_positive_weekly = len(new_weekly_bitcoin_df.loc[new_weekly_bitcoin_df.weekly_return>0]) / 312

weekly_eth_df = eth_df[eth_df.date.dt.weekday == 6]
weekly_eth_df['weekly_return'] = weekly_eth_df['usd_per_token'].pct_change(1)
new_weekly_eth_df = weekly_eth_df.reset_index()[1:]
eth_mean_weekly = statistics.mean(new_weekly_eth_df.weekly_return)
eth_sd_weekly = statistics.stdev(new_weekly_eth_df.weekly_return)
eth_t_stat_weekly = eth_mean_weekly / (eth_sd_weekly/math.sqrt(312))
eth_sharpe_weekly = eth_mean_weekly / eth_sd_weekly
eth_skewness_weekly = skew(new_weekly_eth_df.weekly_return)
eth_kurt_weekly = kurtosis(new_weekly_eth_df.weekly_return,fisher = False)
eth_positive_weekly = len(new_weekly_eth_df.loc[new_weekly_eth_df.weekly_return>0]) / 312

cmkt_mean_weekly = statistics.mean(new_weekly_df.mcap_r_t)
cmkt_sd_weekly = statistics.stdev(new_weekly_df.mcap_r_t)
cmkt_t_stat_weekly = cmkt_mean_weekly / (cmkt_sd_weekly/math.sqrt(313))
cmkt_sharpe_weekly = cmkt_mean_weekly / cmkt_sd_weekly
cmkt_skewness_weekly = skew(new_weekly_df.mcap_r_t)
cmkt_kurt_weekly = kurtosis(new_weekly_df.mcap_r_t,fisher = False)
cmkt_positive_weekly = len(new_weekly_df.loc[new_weekly_df.mcap_r_t>0]) / 313

weekly_stats_df = pd.DataFrame(data={"weekly":("CMKT","Bitcoin","Ethereum","Stock"),
                                    "mean":(cmkt_mean_weekly,bitcoin_mean_weekly,eth_mean_weekly,VTI_mean_weekly),
                                    "sd":(cmkt_sd_weekly,bitcoin_sd_weekly,eth_sd_weekly,VTI_sd_weekly),
                                    "t_stat":(cmkt_t_stat_weekly,bitcoin_t_stat_weekly,eth_t_stat_weekly,VTI_t_stat_weekly),
                                    "sharpe":(cmkt_sharpe_weekly,bitcoin_sharpe_weekly,eth_sharpe_weekly,VTI_sharpe_weekly),
                                    "skewness":(cmkt_skewness_weekly,bitcoin_skewness_weekly,eth_skewness_weekly,VTI_skewness_weekly),
                                    "kurtosis":(cmkt_kurt_weekly,bitcoin_kurt_weekly,eth_kurt_weekly,VTI_kurt_weekly),
                                    "% > 0":(cmkt_positive_weekly,bitcoin_positive_weekly,eth_positive_weekly,VTI_positive_weekly)})

In [ ]:
# MIGHT NEED TO TAKE A LOOK INTO THIS as there are some month-end dates that falls in weekends / holidays and not sure if it will cause a material change to the descriptive statistics of the VTI stock.
new_VTI_df['month_end'] = \
pd.to_datetime(new_VTI_df.Date).dt.is_month_end
VTI_monthly_df = new_VTI_df[new_VTI_df['month_end'] == 1 ]
VTI_monthly_df['monthly_return'] = VTI_monthly_df['Price'].pct_change(1)
new_VTI_monthly_df = VTI_monthly_df.reset_index()[1:]

VTI_mean_monthly = statistics.mean(new_VTI_monthly_df.monthly_return)
VTI_sd_monthly = statistics.stdev(new_VTI_monthly_df.monthly_return)
VTI_t_stat_monthly = VTI_mean_monthly / (VTI_sd_monthly/math.sqrt(49))
VTI_sharpe_monthly = VTI_mean_monthly / VTI_sd_monthly
VTI_skewness_monthly = skew(new_VTI_monthly_df.monthly_return)
VTI_kurt_monthly = kurtosis(new_VTI_monthly_df.monthly_return,fisher = False)
VTI_positive_monthly = len(new_VTI_monthly_df.loc[new_VTI_monthly_df.monthly_return>0]) / 49

bitcoin_df['month_end'] = \
pd.to_datetime(bitcoin_df.date).dt.is_month_end
monthly_bitcoin_df = bitcoin_df[bitcoin_df['month_end'] == 1]
monthly_bitcoin_df['monthly_returns'] = monthly_bitcoin_df['usd_per_token'].pct_change(1)
new_monthly_bitcoin_df = monthly_bitcoin_df.reset_index()[1:]
bitcoin_mean_monthly = statistics.mean(new_monthly_bitcoin_df.monthly_returns)
bitcoin_sd_monthly = statistics.stdev(new_monthly_bitcoin_df.monthly_returns)
bitcoin_t_stat_monthly = bitcoin_mean_monthly / (bitcoin_sd_monthly/math.sqrt(71))
bitcoin_sharpe_monthly = bitcoin_mean_monthly / bitcoin_sd_monthly
bitcoin_skewness_monthly = skew(new_monthly_bitcoin_df.monthly_returns)
bitcoin_kurt_monthly = kurtosis(new_monthly_bitcoin_df.monthly_returns,fisher = False)
bitcoin_positive_monthly = len(new_monthly_bitcoin_df.loc[new_monthly_bitcoin_df.monthly_returns>0]) / 71

eth_df['month_end'] = \
pd.to_datetime(eth_df.date).dt.is_month_end
monthly_eth_df = eth_df[eth_df['month_end'] == 1]
monthly_eth_df['monthly_returns'] = monthly_eth_df['usd_per_token'].pct_change(1)
new_monthly_eth_df = monthly_eth_df.reset_index()[1:]
eth_mean_monthly = statistics.mean(new_monthly_eth_df.monthly_returns)
eth_sd_monthly = statistics.stdev(new_monthly_eth_df.monthly_returns)
eth_t_stat_monthly = eth_mean_monthly / (eth_sd_monthly/math.sqrt(71))
eth_sharpe_monthly = eth_mean_monthly / eth_sd_monthly
eth_skewness_monthly = skew(new_monthly_eth_df.monthly_returns)
eth_kurt_monthly = kurtosis(new_monthly_eth_df.monthly_returns,fisher = False)
eth_positive_monthly = len(new_monthly_eth_df.loc[new_monthly_eth_df.monthly_returns>0]) / 71

cmkt_mean_monthly = statistics.mean(new_monthly_df.mcap_r_t)
cmkt_sd_monthly = statistics.stdev(new_monthly_df.mcap_r_t)
cmkt_t_stat_monthly = cmkt_mean_monthly / (cmkt_sd_monthly/math.sqrt(72))
cmkt_sharpe_monthly = cmkt_mean_monthly / cmkt_sd_monthly
cmkt_skewness_monthly = skew(new_monthly_df.mcap_r_t)
cmkt_kurt_monthly = kurtosis(new_monthly_df.mcap_r_t,fisher = False)
cmkt_positive_monthly = len(new_monthly_df.loc[new_monthly_df.mcap_r_t>0]) / 72

monthly_stats_df = pd.DataFrame(data={"monthly":("CMKT","Bitcoin","Ethereum","Stock"),
                                    "mean":(cmkt_mean_monthly,bitcoin_mean_monthly,eth_mean_monthly,VTI_mean_monthly),
                                    "sd":(cmkt_sd_monthly,bitcoin_sd_monthly,eth_sd_monthly,VTI_sd_monthly),
                                    "t_stat":(cmkt_t_stat_monthly,bitcoin_t_stat_monthly,eth_t_stat_monthly,VTI_t_stat_monthly),
                                    "sharpe":(cmkt_sharpe_monthly,bitcoin_sharpe_monthly,eth_sharpe_monthly,VTI_sharpe_monthly),
                                    "skewness":(cmkt_skewness_monthly,bitcoin_skewness_monthly,eth_skewness_monthly,VTI_skewness_monthly),
                                    "kurtosis":(cmkt_kurt_monthly,bitcoin_kurt_monthly,eth_kurt_monthly,VTI_kurt_monthly),
                                    "% > 0":(cmkt_positive_monthly,bitcoin_positive_monthly,eth_positive_monthly,VTI_positive_monthly)})

In [250]:

fp = '../4-output/descriptive_stat.xlsx'

with pd.ExcelWriter(fp, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
    daily_stats_df.to_excel(writer,sheet_name='daily')
    weekly_stats_df.to_excel(writer,sheet_name='weekly')
    monthly_stats_df.to_excel(writer,sheet_name='monthly')
    Extreme_events_df.to_excel(writer,sheet_name='extreme')
